## 雪球评论爬虫

Alex

Created on: 08/07/2021

In [1]:
import json                     # 爬取结果为JSON格式，用这个进行格式化
import os                       # 转换工作路径
import sys                      # 获取当前python文件所在路径
import time                     # 计时

import pandas as pd
import requests                 # 也可以选用selenium，但是requests代码好写易看一点
from bs4 import BeautifulSoup   # 用于格式化网页CSS结构

key_word        =   '伊利股份'      # key_word 是要查询的内容，可以是股票名，股票代码，行业名称
comments_num    =   1000          # 要爬去的评论数。最新的N条

# 爬取的结果是JSON格式转为Dict，需要进行二次转换
text_keys       =   ['title','view_count','like_count']
user_info_keys  =   ['description','screen_name',
                    'followers_count','friends_count','gender','province']
df_columns      =   ['text','created_at'] + text_keys + user_info_keys

os.chdir(sys.path[0])   # 更改工作路径，方便输出结果
tic=time.time()         # 计时


page_num=comments_num//10+1     # 雪球每页显示十个评论
print('%s comments to crawl, script started!'%comments_num)

s=requests.session()            # 爬虫主体对象
# 雪球反爬虫通过监察访问单位的headers来进行筛查，需要加上User-Agent和Cookie
print('Original header')
print(s.headers) # 运行这个可以看到没有更改headers的时候，User-Agent是python

s.headers.update({'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36'})
## 改成自己的cookie
s.headers.update({'Cookie': 'YOUR_COOKIE'})
# print(s.headers) 

1000 comments to crawl, script started!
Original header
{'User-Agent': 'python-requests/2.24.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}


In [2]:
def get_one_page(key_word,page_index):
    """
    获取单一页面评论
    """
    # 雪球评论页面采用动态加载，分析xhr文件发现，每次加载均访问这个query
    # 返回包含下十个comments的json文件，这个url主要部分是q=搜索内容以及page=页码
    # 更改count内容并不能增加每页显示的评论数
    url='https://xueqiu.com/query/v1/search/status.json?sortId=1&q=%s&count=20&page=%s'%(key_word,page_index)
    for iii in range(10):
        try:
            r=s.get(url).text           # 调用爬虫主体对象访问目标url并返回网站text
            res_temp=json.loads(r)      # 这个text结构式json，格式化为dict方便处理
            res_temp=res_temp['list']   # 观察发现只有list中的内容是我们需要的
            break
        except:
            # 有的时候会遇到反爬虫现象，简单的访问过于频繁以及次数过多，歇几秒就行了
            print('comments #%s failed x%s, sleep for 3s, timer: %ss'%(len(res_raw),iii,round(time.time()-tic,2)))
            time.sleep(3)
    
    if page_index%5==0: 
        # 每爬5页print一下进度
        print('comments got %s/%s, timer: %ss'%(len(res_raw),comments_num,round(time.time()-tic,2)))
    time.sleep(0.3) # 每次访问都歇一歇，免得被block

    return res_temp

def single_processor(dct):
    """
    每个comment的结果仍为json（或dict）我们需要格式化一下，转换成Excel方便分析
    """
    res=[]                              # 空list，用于存储处理结果

    text=dct['text']                    # 获取text键，这是评论的内容
    text=BeautifulSoup(text)            # 这个内容是html格式，需要用beautiful soup进行格式化
    
    text=text.find_all('body')[0]       # body 标签下的是我们要的东西
    text=text.get_text()                # 获取该结构下的文本
    text=text.replace('\xa0','')        # 有一些错误代码，删除
    res.append(text)                    

    # json套json，这部分处理获取comments的发表时间，有的评论没有这一些，记为none
    try:    res.append(eval(dct['trackJson'])['created_at'])
    except: res.append('none')

    # 将comments标题，浏览数，点赞数提取
    for key in text_keys:
        res.append(dct[key])

    # 提取作者信息，简介，用户名等
    user=dct['user']
    for key in user_info_keys:
        try:res.append(user[key])
        except:res.append('none')
    return res


res_raw=[]
for i in range(page_num):   # 依次访问每一页
    res_raw.extend(get_one_page(key_word,i))
print(len(res_raw),'comments got!')


res=list(map(lambda x: single_processor(x),res_raw))    # 分布式处理每一个comments
res=pd.DataFrame(res,columns=df_columns)                # 将结果存储在dataframe
res.rename(columns={'description':'user_description',   # 列重命名
                    'screen_name':'user_name'},
           inplace=True)

res

comments got 0/1000, timer: 4.96s
comments got 50/1000, timer: 9.97s
comments got 100/1000, timer: 16.16s
comments got 148/1000, timer: 22.46s
comments got 198/1000, timer: 27.17s
comments got 248/1000, timer: 31.98s
comments got 298/1000, timer: 39.58s
comments got 347/1000, timer: 47.27s
comments got 397/1000, timer: 54.81s
comments got 447/1000, timer: 59.22s
comments got 496/1000, timer: 66.52s
comments got 546/1000, timer: 73.35s
comments got 596/1000, timer: 81.13s
comments #626 failed x0, sleep for 3s, timer: 85.12s
comments got 646/1000, timer: 90.96s
comments got 695/1000, timer: 99.05s
comments got 745/1000, timer: 105.84s
comments got 795/1000, timer: 112.57s
